In [1]:
import sys
import os

# プロジェクトルートを Python パスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import numpy as np
from graph.prior import GaussianPrior
from graph.measurement import GaussianMeasurement
from core.uncertain_array import UncertainArray as UA

In [3]:
# 1. Prior → X
X = GaussianPrior(shape=(4, 4), mean=0, var=1.0, seed=42)

# 2. X → Measurement（観測値 y）
obs_data = np.ones((4, 4), dtype=np.complex128)
meas = GaussianMeasurement(X, obs_data, var=0.25)

# 3. メッセージ伝播
X.parent.forward()   # Prior → X
X.forward()          # X → Measurement
meas.backward()      # Measurement → X
X.compute_belief()   # Belief 更新

# 4. 結果確認
print("Mean:\n", X.belief.data)
print("Precision:\n", X.belief.precision)

Mean:
 [[0.8430935 +0.05214924j 0.65292404-0.13560648j 0.90612983+0.12423163j
  0.93301594-0.00706059j]
 [0.52408196-0.02614349j 0.61584401-0.09629798j 0.81807936+0.17289345j
  0.75527654-0.02185377j]
 [0.79762396-0.0605747j  0.67936137-0.0497992j  0.92436565+0.07527989j
  0.90999639+0.0516816j ]
 [0.80933815+0.05836921j 0.95941598+0.06092729j 0.86611581+0.30287471j
  0.67847769-0.05747576j]]
Precision:
 [[5. 5. 5. 5.]
 [5. 5. 5. 5.]
 [5. 5. 5. 5.]
 [5. 5. 5. 5.]]


In [4]:
import numpy as np

# ライブラリのインポート
from graph.prior import GaussianPrior
from graph.measurement import MaskedGaussianMeasurement
from core.uncertain_array import UncertainArray as UA
from core.linalg_utils import random_binary_mask

# 1. Prior によって Wave を生成
X = GaussianPrior(shape=(8, 8), mean=0, var=1.0, seed=42)

# 2. ランダムマスクを生成し、観測値を作る（全て 1+0j にする）
mask = random_binary_mask((8, 8), subsampling_rate=0.5, seed=1)
obs_data = np.ones((8, 8), dtype=np.complex128)

# 3. マスク付き GaussianMeasurement を構築
meas = MaskedGaussianMeasurement(X, observed_data=obs_data, var=0.25, mask=mask)

# 4. メッセージ伝播を実行
X.parent.forward()   # Prior → X
X.forward()          # X → Measurement
meas.backward()      # Measurement → X
X.compute_belief()   # Belief 更新

# 5. 結果表示
print("Mask (True = observed):\n", mask.astype(int))
print("Belief precision:\n", X.belief.precision)
print("Belief mean:\n", X.belief.data)


Mask (True = observed):
 [[0 1 0 1 0 1 0 1]
 [0 1 1 1 0 1 0 0]
 [1 1 0 1 0 0 1 1]
 [1 1 1 1 0 0 1 1]
 [0 1 1 1 1 1 0 0]
 [0 0 0 0 1 0 0 1]
 [0 1 1 0 1 1 0 0]
 [0 1 0 1 0 0 0 0]]
Belief precision:
 [[2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2.]]
Belief mean:
 [[ 0.17955626+0.41909428j  0.54410336+0.09349687j  0.44220761-0.20548822j
   0.77751328-0.05448868j -1.14965851+0.50556714j  0.51320334-0.02254543j
   0.07533068-0.75170537j  0.62939712-0.13355918j]
 [-0.00990018-0.54179246j  0.56613448+0.05859096j  0.77030471+0.01678503j
   0.75833033+0.08137448j  0.03890896-0.25176104j  0.79951332+0.01868408j
   0.27548252+0.36863267j -0.50634294-0.1822842j ]
 [ 0.71012436+0.05383148j  0.55366127-0.07800872j  0.5176318 -0.21393153j
   0.66078284-0.04498824j -0.10893119-0.70465527j -0.40124158+0.28695129j
   0.81074455-0.0553196j  